<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2025/aoc2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import requests

def get_aoc_input(year, day):
    """
    Retrieves the Advent of Code input for a given year and day.

    Args:
        year (int): The Advent of Code year.
        day (int): The Advent of Code day.

    Returns:
        str: The input data as a string, or None if retrieval fails.
    """
    session_cookie = input("Please enter your Advent of Code session cookie: ")
    if not session_cookie:
        print("Session cookie is required to retrieve Advent of Code input.")
        return None

    headers = {
        'Cookie': f'session={session_cookie}'
    }

    # Advent of Code typically doesn't provide a direct URL for 'test' input.
    # The standard input URL is for the puzzle input.
    # For test inputs, users usually copy-paste from the problem description.
    # This function will retrieve the *puzzle* input. If you need test input
    # you will generally have to manually copy it from the problem page.
    url = f"https://adventofcode.com/{year}/day/{day}/input"

    print(f"Attempting to retrieve Advent of Code {year} Day {day} puzzle input from {url}...")
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.text.strip()
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Error: Input not found for Advent of Code {year} Day {day}. Check if the year/day is valid or if it's released yet. (Status Code: 404)")
        elif e.response.status_code == 400:
            print(f"Error: Bad request. This often indicates an invalid session cookie. (Status Code: 400)")
        else:
            print(f"HTTP error occurred: {e}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the request: {e}")
        return None

In [26]:
import requests
from bs4 import BeautifulSoup

def get_aoc_test_input(year, day):
    """
    Attempts to retrieve the Advent of Code test input for a given year and day. Returns first candidate on problem page.

    Args:
        year (int): The Advent of Code year.
        day (int): The Advent of Code day.

    Returns:
        str: The input data as a string, or None if retrieval fails.
    """
    session_cookie = input("Please enter your Advent of Code session cookie: ")
    if not session_cookie:
        print("Session cookie is required to retrieve Advent of Code input.")
        return None

    headers = {
        'Cookie': f'session={session_cookie}'
    }

    # Construct the URL for the problem page
    problem_url = f"https://adventofcode.com/{year}/day/{day}"

    print(f"Attempting to retrieve Advent of Code {year} Day {day} problem page from {problem_url}...")

    try:
        # Make the HTTP GET request
        response = requests.get(problem_url, headers=headers)
        response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
        problem_page_html = response.text
        print("Successfully retrieved problem page HTML.")
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Error: Problem page not found for Advent of Code {year} Day {day}. Check if the year/day is valid. (Status Code: 404)")
        elif e.response.status_code == 400:
            print(f"Error: Bad request. This often indicates an invalid session cookie. (Status Code: 400)")
        else:
            print(f"HTTP error occurred: {e}")
        problem_page_html = None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the request: {e}")
        problem_page_html = None

    if problem_page_html:
        print(f"\nFirst 500 characters of retrieved HTML:\n---\n{problem_page_html[:500]}\n---")
    else:
        print(f"Failed to retrieve Advent of Code {year} Day {day} problem page HTML.")
        return None

    soup = BeautifulSoup(problem_page_html, 'html.parser')

    # Find all <code> tags
    code_tags = soup.find_all('code')

    print(f"Found {len(code_tags)} <code> tags. Inspecting content for test input:")
    print("---")

    test_input_candidates = []
    for i, code_tag in enumerate(code_tags):
        # Advent of Code usually puts test inputs in <pre><code>...</code></pre> blocks
        # or sometimes directly in <code> tags within paragraphs.
        # We'll collect all of them and let the user visually identify.
        content = code_tag.get_text(strip=True)
        if content:
            test_input_candidates.append(content)
            print(f"Candidate {i+1}:\n{content}\n")

    if test_input_candidates:
        print("Test input candidates extracted. Please review them to identify the actual test input.")
    else:
        print("No text content found within <code> tags. Test input might be in another tag or format.")
        return None

    test_input = test_input_candidates[0] # Assuming Candidate 1 is the correct test input

    print("Identified Test Input:")
    print("---------------------")
    print(test_input)
    print("---------------------")

    return test_input